<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-the-data" data-toc-modified-id="Get-the-data-1">Get the data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#A-look-at-the-full-dataset" data-toc-modified-id="A-look-at-the-full-dataset-1.0.1">A look at the full dataset</a></span></li></ul></li></ul></li><li><span><a href="#Examine-the-data" data-toc-modified-id="Examine-the-data-2">Examine the data</a></span></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# from scipy.stats import pearsonr
import scipy.stats as stats 
import statsmodels.api as sm
from statsmodels.formula.api import logit
from statsmodels.formula.api import ols

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Get the data

### A look at the full dataset

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/NLP_data/advertising.csv')
df_original = pd.read_csv('/Users/jimcody/Documents/GitHub/cdc_stats/data/framingham heart study-M.csv')
df_original.columns= df.columns.str.lower()


In [ ]:
df = df_original
df.head()

# Examine the data

In [ ]:
print('Shape: ',df.shape)
df.info()

In [ ]:
# Change to string
df['snack'] = df['snack'].astype(str)
df['lpa'] = df['lpa'].astype(str)

# Change from codes
#df['snack'] = df['snack'].map({1: 'cookies', 2: 'ice cream', 3: 'chips', 4: 'candy', 5: 'healthy'})
#df['lpa'] = df['lpa'].map({1: 'very little', 2: 'some', 3: 'moderate', 4: 'daily', 5: 'two a day'})

In [ ]:
df.loc[df.duplicated()]

In [ ]:
df.isnull().sum()

In [ ]:
df.education.fillna( df.education.mean(),inplace=True )
df.cigsperday.fillna( df.cigsperday.mean(),inplace=True )
df.totchol.fillna( df.totchol.mean(),inplace=True )
df.bmi.fillna( df.bmi.mean(),inplace=True )
df.glucose.fillna( df.glucose.mean(),inplace=True )

In [ ]:
df.bpmeds.unique()

In [ ]:
df.bpmeds.fillna( 0,inplace=True )
df['bpmeds'] = df['bpmeds'].astype(str)

# Can be written as
#df.bpmeds=df.bpmeds.astype(str)

In [ ]:
df.columns

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(df, vars = ['age','cigsperday','totchol','sysbp','diabp','bmi','heartrate','glucose'])

In [ ]:
corr = df.corr()['tenyearchd']
corr

In [32]:
import statsmodels.formula.api as smf
formula = 'tenyearchd ~ glucose'
glm_model = smf.glm(formula = formula, data=df, family=sm.families.Binomial())
glm_result = glm_model.fit()
print(glm_result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             tenyearchd   No. Observations:                 4240
Model:                            GLM   Df Residuals:                     4238
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1782.8
Date:                Thu, 09 Dec 2021   Deviance:                       3565.6
Time:                        06:26:14   Pearson chi2:                 4.25e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.6020      0.140    -18.617      0.0

In [ ]:
np.exp(glm_result.params)

For every one unit increase in glucose measure, the is a 1% increase in the odds of developing CHD within 10 years.

In [38]:
import statsmodels.formula.api as smf
formula = 'tenyearchd ~ glucose+age+male+prevalentstroke+totchol+sysbp'
glm2_model = smf.glm(formula = formula, data=df, family=sm.families.Binomial())
glm2_result = glm2_model.fit()
print(glm2_result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             tenyearchd   No. Observations:                 4240
Model:                            GLM   Df Residuals:                     4233
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1622.6
Date:                Thu, 09 Dec 2021   Deviance:                       3245.1
Time:                        06:38:53   Pearson chi2:                 4.27e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -8.3395      0.422    -

In [33]:
logit_model = logit('tenyearchd ~ glucose', df ).fit()
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.420470
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             tenyearchd   No. Observations:                 4240
Model:                          Logit   Df Residuals:                     4238
Method:                           MLE   Df Model:                            1
Date:                Thu, 09 Dec 2021   Pseudo R-squ.:                 0.01291
Time:                        06:26:19   Log-Likelihood:                -1782.8
converged:                       True   LL-Null:                       -1806.1
Covariance Type:            nonrobust   LLR p-value:                 8.604e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.6020      0.140    -18.617      0.000      -2.876      -2.328
glucose        0.0105      0.

- Pseudo- Rsq: substitute for Rsq
- Log-likelihood: F statistic
- LL-Null: F-probability

In [36]:
logit_model2 = logit('tenyearchd ~ glucose+age+bmi+male+prevalentstroke+prevalenthyp+diabetes+totchol+sysbp', df ).fit()
print(logit_model2.summary())

Optimization terminated successfully.
         Current function value: 0.382259
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             tenyearchd   No. Observations:                 4240
Model:                          Logit   Df Residuals:                     4230
Method:                           MLE   Df Model:                            9
Date:                Thu, 09 Dec 2021   Pseudo R-squ.:                  0.1026
Time:                        06:29:11   Log-Likelihood:                -1620.8
converged:                       True   LL-Null:                       -1806.1
Covariance Type:            nonrobust   LLR p-value:                 2.480e-74
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -7.8417      0.540    -14.533      0.000      -8.899      -6.784
glucose       

In [37]:
logit_model3 = logit('tenyearchd ~ glucose+age+male+prevalentstroke+totchol+sysbp', df ).fit()
print(logit_model3.summary())

Optimization terminated successfully.
         Current function value: 0.382678
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             tenyearchd   No. Observations:                 4240
Model:                          Logit   Df Residuals:                     4233
Method:                           MLE   Df Model:                            6
Date:                Thu, 09 Dec 2021   Pseudo R-squ.:                  0.1016
Time:                        06:30:12   Log-Likelihood:                -1622.6
converged:                       True   LL-Null:                       -1806.1
Covariance Type:            nonrobust   LLR p-value:                 3.279e-76
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -8.3395      0.422    -19.740      0.000      -9.167      -7.511
glucose       